# Q1. Contraception Use Analysis

#### 0. Load the data from ../data/other/contraception.csv using the code below.

In [84]:
import pandas as pd
import numpy as np
from scipy.stats import chi2

df_00 = pd.read_csv("contraception.csv")
df_00["ageRange"] = np.where(
    df_00["ageRange"]=="<20",
    "00-20",
    df_00["ageRange"]
)
df_00

,ageRange,noUse,yesUse,notMore,highEd
0,00-20,213,54,NO,YES
1,00-20,200,54,NO,YES
2,00-20,52,13,NO,NO
3,00-20,45,6,YES,YES
4,00-20,21,4,YES,NO
5,00-20,15,6,YES,NO
6,20-29,165,52,NO,YES
7,20-29,152,53,NO,YES
8,20-29,66,29,YES,YES
9,20-29,60,12,NO,NO


#### 1. Print out the data types for each column.

In [87]:
df_00.dtypes

ageRange    object
noUse        int64
yesUse       int64
notMore     object
highEd      object
dtype: object

#### 2. Create dummies for ageRange, notMore, and highEd. Drop the first category for each variable.

In [89]:
import numpy as np

df_w_dummies = pd.get_dummies(
    df_00,
    columns=[
        'ageRange',
        'notMore',
        'highEd'
        ],
    drop_first=True
    )

In [91]:
# Get the dummy columns where the type is 'bool'
dummy_columns = df_w_dummies.select_dtypes(include='bool').columns
# Convert type from 'bool' to 'int'
df_w_dummies[dummy_columns] = df_w_dummies[dummy_columns].astype(int)
df_w_dummies

,noUse,yesUse,ageRange_20-29,ageRange_30-39,ageRange_40-49,notMore_YES,highEd_YES
0,213,54,0,0,0,0,1
1,200,54,0,0,0,0,1
2,52,13,0,0,0,0,0
3,45,6,0,0,0,1,1
4,21,4,0,0,0,1,0
5,15,6,0,0,0,1,0
6,165,52,1,0,0,0,1
7,152,53,1,0,0,0,1
8,66,29,1,0,0,1,1
9,60,12,1,0,0,0,0


#### 3. Create a table with basic statistics.

In [94]:
df_w_dummies.describe().T

,count,mean,std,min,25%,50%,75%,max
noUse,24.0,67.250000,62.301685,4.0,16.5,48.5,87.50,213.0
yesUse,24.0,32.291667,26.260367,4.0,9.0,29.0,52.25,83.0
ageRange_20-29,24.0,0.250000,0.442326,0.0,0.0,0.0,0.25,1.0
ageRange_30-39,24.0,0.208333,0.414851,0.0,0.0,0.0,0.00,1.0
ageRange_40-49,24.0,0.291667,0.464306,0.0,0.0,0.0,1.00,1.0
notMore_YES,24.0,0.500000,0.510754,0.0,0.0,0.5,1.00,1.0
highEd_YES,24.0,0.500000,0.510754,0.0,0.0,0.5,1.00,1.0


#### 4. Estimate an intercept only binomial regression model of the use of contraceptives.

In [179]:
import statsmodels.api as sm

df_w_dummies['yesUse'] = pd.to_numeric(df_w_dummies['yesUse'])
df_w_dummies['noUse'] = pd.to_numeric(df_w_dummies['noUse'])

# Drop rows with NaN values in 'yesUse' or 'noUse'
df_w_dummies = df_w_dummies.dropna(subset=['yesUse', 'noUse'])

# Create the list of tuples (dependent variable)
dep_var = list(zip(df_w_dummies['yesUse'], df_w_dummies['noUse']))

# Fit the logistic regression model
intercept_model = sm.GLM(
    dep_var,
    sm.add_constant(np.zeros(len(df_w_dummies))),  # Intercept-only model
    family=sm.families.Binomial(link=sm.families.links.logit())
).fit()

# Print the summary of the model
print(intercept_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           ['y1', 'y2']   No. Observations:                   24
Model:                            GLM   Df Residuals:                       23
Model Family:                Binomial   Df Model:                            0
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -178.93
Date:                Mon, 25 Nov 2024   Deviance:                       252.85
Time:                        11:27:37   Pearson chi2:                     255.
No. Iterations:                     4   Pseudo R-squ. (CS):         -7.105e-15
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7336      0.044    -16.786      0.0

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The logit link alias is deprecated. Use Logit instead. The logit link alias will be removed after the 0.15.0 release.
  warnings.warn(


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

#### 5. Using the estimate for the intercept, compute the probability of contraceptive usage.


In [100]:
intercept_value = mod1.params[0]

predicted_probability = 1 / (1 + np.exp(-intercept_value))

predicted_probability

0.3244035161155296

In [188]:
predicted_probability = 1 / (1 + np.exp(1.58045))

predicted_probability

0.17073176048988276

#### 6. Estimate the model using all available variables. Interpret the results.

In [103]:
df_w_dummies.head()

,noUse,yesUse,ageRange_20-29,ageRange_30-39,ageRange_40-49,notMore_YES,highEd_YES
0,213,54,0,0,0,0,1
1,200,54,0,0,0,0,1
2,52,13,0,0,0,0,0
3,45,6,0,0,0,1,1
4,21,4,0,0,0,1,0


In [107]:
df_w_dummies['yesUse'] = pd.to_numeric(df_w_dummies['yesUse'])
df_w_dummies['noUse'] = pd.to_numeric(df_w_dummies['noUse'])

# Drop rows with NaN values in 'yesUse' or 'noUse'
df_w_dummies = df_w_dummies.dropna(subset=['yesUse', 'noUse'])

X = df_w_dummies.drop(columns=['noUse', 'yesUse'])

# Fit the logistic regression model
full_model = sm.GLM(
    dep_var,
    sm.add_constant(X),  
    family=sm.families.Binomial(link=sm.families.links.logit())
).fit()

print(full_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           ['y1', 'y2']   No. Observations:                   24
Model:                            GLM   Df Residuals:                       18
Model Family:                Binomial   Df Model:                            5
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -83.024
Date:                Mon, 25 Nov 2024   Deviance:                       61.044
Time:                        09:17:24   Pearson chi2:                     55.5
No. Iterations:                     4   Pseudo R-squ. (CS):             0.9997
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.7212      0.138    -12.

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The logit link alias is deprecated. Use Logit instead. The logit link alias will be removed after the 0.15.0 release.
  warnings.warn(


#### 7. Manually compute the deviation and the corresponding p-value. Interpret the results.
The result below shows that the residual deviance is significantly different from zero, and the P-Value is extremely small (< 0.05). This means that the saturated model (perfect fit) is significantly better than the current model.

In [145]:
y = df_w_dummies['yesUse']  # Observed counts
n = df_w_dummies['yesUse'] + df_w_dummies['noUse']  # Total number of trials
fitted_values = full_model.fittedvalues  # Predicted values

yh = n * fitted_values

# Compute the deviance
deviance_full = 2 * (
    np.sum(np.where(y != 0, y * np.log(y / yh), 0)) +
    np.sum(np.where(n != y, (n - y) * (np.log((n - y) / (n - yh))), 0))
)

# Compute the residual degrees of freedom
n = len(df_w_dummies)  # Total number of observations
k = full_model.df_model + 1  # Number of predictors + intercept (already available in full_model)

df_resid_full = n - k

# Compute the P-value
p_value_full = chi2.sf(deviance_full, df=df_resid_full)

print(f"Residual Deviance of Full Model: {deviance_full}")
print(f"Residual Degrees of Freedom of Full Model: {df_resid_full}")
print(f"P-value of Full Model: {p_value_full}")

Residual Deviance: 61.0437019052926
Residual Degrees of Freedom: 18
P-value: 1.3860653488596432e-06


#### 8. Compare the model with all variables against the intercept only model. Interpret the results.
The residual deviance for the intercept-only model (252.8514) is much larger than the deviance for the full model (61.0437). This indicates that the full model explains the data much better. Moreover, the P-Value is smaller in the intercept only model, which shows that the full model is an improvement relative to the intercept only model

In [153]:
# Deviance and residual df of the intercept model
deviance_intercept = intercept_model.deviance
df_resid_intercept = intercept_model.df_resid

# Compute the difference in degrees of freedom
df_diff = df_resid_full - df_resid_intercept

# Compute the p-value
p_value_intercept = chi2.sf(deviance_intercept, df_resid_intercept)

# Print the results
print(f"Residual Deviance of Intercept Only Model: {deviance_intercept}")
print(f"Residual Degrees of Freedom of Intercept Only Model: {df_resid_intercept}")
print(f"P-value of Intercept Only Model: {p_value_intercept}")

Residual Deviance of Intercept Only Model: 252.85141999517788
Residual Degrees of Freedom of Intercept Only Model: 23
P-value of Intercept Only Model: 1.3337677961739572e-40


#### 9. Create a new model with interaction dummies for ageRange and notMore. Interpret the results.

In [161]:
# Create interaction terms
df_w_dummies['ageRange_20-29 * notMore_YES'] = df_w_dummies['ageRange_20-29'] * df_w_dummies['notMore_YES']
df_w_dummies['ageRange_30-39 * notMore_YES'] = df_w_dummies['ageRange_30-39'] * df_w_dummies['notMore_YES']
df_w_dummies['ageRange_40-49 * notMore_YES'] = df_w_dummies['ageRange_40-49'] * df_w_dummies['notMore_YES']
df_w_dummies.head()

,noUse,yesUse,ageRange_20-29,ageRange_30-39,ageRange_40-49,notMore_YES,highEd_YES,ageRange_20-29 * notMore_YES,ageRange_30-39 * notMore_YES,ageRange_40-49 * notMore_YES
0,213,54,0,0,0,0,1,0,0,0
1,200,54,0,0,0,0,1,0,0,0
2,52,13,0,0,0,0,0,0,0,0
3,45,6,0,0,0,1,1,0,0,0
4,21,4,0,0,0,1,0,0,0,0


In [167]:
X_interaction = df_w_dummies.drop(columns=['noUse', 'yesUse'])

# Fit the logistic regression model
interaction_model = sm.GLM(
    dep_var,
    sm.add_constant(X_interaction),  
    family=sm.families.Binomial(link=sm.families.links.logit())
).fit()

print(interaction_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           ['y1', 'y2']   No. Observations:                   24
Model:                            GLM   Df Residuals:                       15
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -72.636
Date:                Mon, 25 Nov 2024   Deviance:                       40.267
Time:                        10:47:27   Pearson chi2:                     38.9
No. Iterations:                     5   Pseudo R-squ. (CS):             0.9999
Covariance Type:            nonrobust                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The logit link alias is deprecated. Use Logit instead. The logit link alias will be removed after the 0.15.0 release.
  warnings.warn(


#### 10. Manually compute the deviation and the corresponding p-value. Interpret the results.

In [169]:
y = df_w_dummies['yesUse']  # Observed counts
n = df_w_dummies['yesUse'] + df_w_dummies['noUse']  # Total number of trials
fitted_values = interaction_model.fittedvalues  # Predicted values

yh = n * fitted_values

# Compute the deviance
deviance_interaction = 2 * (
    np.sum(np.where(y != 0, y * np.log(y / yh), 0)) +
    np.sum(np.where(n != y, (n - y) * (np.log((n - y) / (n - yh))), 0))
)

# Compute the residual degrees of freedom
n = len(df_w_dummies)  # Total number of observations
k = interaction_model.df_model + 1  # Number of predictors + intercept (already available in full_model)

df_resid_interaction = n - k

# Compute the P-value
p_value_interaction = chi2.sf(deviance_interaction, df=df_resid_interaction)

print(f"Residual Deviance: {deviance_interaction}")
print(f"Residual Degrees of Freedom: {df_resid_interaction}")
print(f"P-value of Full Model: {p_value_interaction}")

Residual Deviance of Full Model: 40.26738302704832
Residual Degrees of Freedom of Full Model: 15
P-value of Full Model: 0.0004131979091986302


#### 11. Compare the model with all variables and the interaction terms against the model with all variables. Interpret the results.

In [181]:
print("<Model with all variables & interaction terms>")
print(f"Residual Deviance: {deviance_interaction}")
print(f"Residual Degrees of Freedom: {df_resid_interaction}")
print(f"P-value: {p_value_interaction}")

print("---------------------------------------------------------------")

print("<Model with all variables>")
print(f"Residual Deviance: {deviance_full}")
print(f"Residual Degrees of Freedom: {df_resid_full}")
print(f"P-value: {p_value_full}")

<Model with all variables & interaction terms>
Residual Deviance: 40.26738302704832
Residual Degrees of Freedom: 15
P-value: 0.0004131979091986302
---------------------------------------------------------------
<Model with all variables>
Residual Deviance: 61.0437019052926
Residual Degrees of Freedom: 18
P-value: 1.3860653488596432e-06


The model with interactions is still significantly different from the saturated model as its P-Value is small (< 0.05).
However, the residual deviance for the model with all variables and the interaction terms model (40.2674) has decreased compared to that of the model with all variables without the interaction terms (61.0437). 
This indicates that the model with interaction terms has improved compared to the model without interaction terms.